#  This project is to create Sparkify key space and NoSQL data models using Cassandra to present data based on the requests below:

1. Show the artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4
2. Show only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
3. Show every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

# Part I. ETL Pipeline for Pre-Processing the Files

#### Import Python packages 

In [34]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [35]:
# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):    
    file_path_list = glob.glob(os.path.join(root,'*'))

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [36]:
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        csvreader = csv.reader(csvfile) 
        next(csvreader)       
        for line in csvreader:
            full_data_rows_list.append(line)
        
# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue # skip artist name is null row
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))

#### check the number of rows in your csv file

In [37]:
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Build the Apache Cassandra tables bases on the query requests. 

#### Creating a Cluster

In [38]:
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])
session = cluster.connect()

#### Create Keyspace

In [39]:
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS sparkify
    WITH REPLICATION = 
    { 'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)

#### Set Keyspace

In [40]:
try:
    session.set_keyspace('sparkify')
except Exception as e:
    print(e)

### Query 1 
#### artist, song title and song's length in the music app history that was heard during sessionId = 338, and itemInSession = 4

In [41]:
# Create table for query 1
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_data_by_session_item (
            session_id INT,
            item_in_session INT,
            artist VARCHAR,
            song VARCHAR,
            length DECIMAL,
            PRIMARY KEY (session_id, item_in_session));
    """)
except Exception as e:
    print(e)

In [42]:
# Insert Data to table song_data_by_session_item
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data_by_session_item (session_id, item_in_session, artist, song, length)"
        query = query + "VALUES(%s, %s, %s, %s, %s)"
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))
        

In [43]:
#Show songs data whose sessionId = 338 and itemInSession = 4 for query 1
query_1 = "select artist, song, length from song_data_by_session_item where session_id = 338 and item_in_session = 4"
results = session.execute(query_1)
for result in results:
    print('artist_name: ',result.artist, ', song: ', result.song, ', length: ', result.length)                 

artist_name:  Faithless , song:  Music Matters (Mark Knight Dub) , length:  495.3073


# Query 2
#### name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182

In [44]:
# Create table for query 2
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS song_data_by_user_session (
            user_id INT,
            session_id INT,
            item_in_session INT,
            artist VARCHAR,
            song VARCHAR,
            first_name VARCHAR,
            last_Name VARCHAR,
            PRIMARY KEY ((user_id, session_id),item_in_session));
    """)
except Exception as e:
    print(e)

In [45]:
# Insert Data to table song_data_by_user_session
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO song_data_by_user_session (user_id, session_id, item_in_session, artist, song, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s, %s, %s, %s)"
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))
        

In [46]:
#Show songs data: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
query_2 = "select artist, song , last_name , first_name from song_data_by_user_session where session_id = 182 and user_id = 10"
results = session.execute(query_2)
for result in results:
    print('artist_name: ',result.artist, ', song: ', result.song, ', user_name: ', result.last_name+' '+result.first_name) 

artist_name:  Down To The Bone , song:  Keep On Keepin' On , user_name:  Cruz Sylvie
artist_name:  Three Drives , song:  Greece 2000 , user_name:  Cruz Sylvie
artist_name:  Sebastien Tellier , song:  Kilometer , user_name:  Cruz Sylvie
artist_name:  Lonnie Gordon , song:  Catch You Baby (Steve Pitron & Max Sanna Radio Edit) , user_name:  Cruz Sylvie


# Query 3
#### every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'

In [47]:
# Create table for query 3, primary_key is a combination of song and user_id as song is not unique and user_id can sort name.
try:
    session.execute("""
        CREATE TABLE IF NOT EXISTS user_data_by_song (
            song VARCHAR,
            user_id INT,
            first_name VARCHAR,
            last_Name VARCHAR,
            PRIMARY KEY (song, user_id));
    """)
except Exception as e:
    print(e)

In [48]:
# Insert Data to table user_data_by_song
file = 'event_datafile_new.csv'
with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        query = "INSERT INTO user_data_by_song (song, user_id, first_name, last_name)"
        query = query + "VALUES(%s, %s, %s, %s)"
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))
        

In [51]:
# Show every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'
query_3 = "select last_name , first_name from user_data_by_song where song = 'All Hands Against His Own'"
results = session.execute(query_3)
for result in results:
    print('user_name: ', result.last_name+' '+result.first_name)                   

user_name:  Lynch Jacqueline
user_name:  Levine Tegan
user_name:  Johnson Sara


### Drop the tables before closing out the sessions

In [32]:
session.execute('drop table song_data_by_session_item')
session.execute('drop table song_data_by_user_session')
session.execute('drop table user_data_by_song')

### Close the session and cluster connection¶

In [33]:
session.shutdown()
cluster.shutdown()